获取历史条目``edit_history.csv``之后，用本程序获取每次编辑后页面的内容

这里爬取了每个页面源码中id为bodyContent的部分，结果保存在webcontent文件夹内

In [3]:
import requests as rq
from bs4 import BeautifulSoup, Comment
import csv
import unicodedata
import numpy
import json

# edit_history.csv保存了所有历史编辑页面的id，通过这个id可以获取到历史页面的链接，也即是urlhead+oldid
# 每次爬取时，先确定要爬取的页面id范围，下面是edit_history.csv的第1到第2个id的示例代码
# 需要修改的地方有：
# 1. start_row, max_row 起止id的索引
# 2. urlhead title修改成目标词条
# 3. proxies修改成自己的代理


urlhead = "https://en.wikipedia.org/w/index.php?title=Donald_Trump&oldid=" # 请修改

s = rq.session()
s.proxies = {"https": "127.0.0.1:1080"}  # 修改成自己的代理
s.keep_alive = False

max_row = 3844#7610 # 请修改
start_row = 3834#3834 # 请修改
with open("../settings.json", "r") as f_set:
    setting_json = f_set.read()
    jstype = json.loads(setting_json)
    start_row = jstype["startid"]
    max_row = jstype["endid"]

with open('edit_history.csv', 'r', encoding='utf-8') as f1:
    rows = csv.reader(f1)
    for row in rows:
        id = int(row[0])
        if id < start_row:
            continue
        if id > max_row:
            print('finish')
            break
        page_oldid = str(row[1])
        print(f'try to get page, id: {id}, url_id: {page_oldid}')
        page_link = urlhead + page_oldid
        try:
            r = s.get(page_link, timeout=60)
            r.encoding = 'utf-8'
            soup = BeautifulSoup(r.text, 'lxml')
            dateDOM = soup.find('span', id='mw-revision-date')
            revdate = dateDOM.get_text()
            with open(f'./content_{id}_{page_oldid}'+'.txt', 'w', encoding='utf-8') as f2:
                body = soup.find('div', id='bodyContent')
                comments = body.findAll(text=lambda text:isinstance(text, Comment))
                [comment.extract() for comment in comments]
                f2.write(f'{str(id)} {str(page_oldid)} {revdate}\n')
                f2.write(str(body))
                print(f'succeed to get page {id}!')
        except:
            print(f'fail to get page {id}!')
            break


try to get page, id: 3854, url_id: 997013342
succeed to get page 3854!
try to get page, id: 3855, url_id: 997012719
succeed to get page 3855!
try to get page, id: 3856, url_id: 997011238
succeed to get page 3856!
finish
